In [ ]:
import seisbench.data as sbd

import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch

from dataset_creation.utils import create_loader_by_phase_and_length
from snr.calc_snr import calc_batch_snr, get_high_snr_traces, str2snr_strategy
from utils.common import assert_path_exists

# Extract High SNR Traces from Dataset

## Requirements and Configuration

What is your user root directory?  (`/home/<username>/` on linux machines e.g.)

In [ ]:
USER_ROOT_DIR='/home/moshe/'
assert_path_exists(USER_ROOT_DIR)
USER_ROOT_DIR

What is the root folder of your datasets?

In [ ]:
DATASETS_ROOT_DIR= os.path.join(USER_ROOT_DIR,'datasets/GFZ/')
assert_path_exists(DATASETS_ROOT_DIR)
DATASETS_ROOT_DIR

The extracted high SNR traces should be taken from the following dataset ('ethz', 'geofon'):

In [ ]:
DATASETS_ORIGINS = ['ethz', 'geofon']
dataset_origin = 'ethz'
assert dataset_origin in DATASETS_ORIGINS, f'Expected dataset one of {DATASETS_ORIGINS}. Got {dataset_origin}.'

Which partition of the data to use (train, dev, test)?

In [ ]:
PARTITION_TO_USE = 'test'

In [ ]:
SBD_CLASSES={'ethz':sbd.ETHZ, 'geofon':sbd.GEOFON}
SBD_CLASS=SBD_CLASSES[dataset_origin]
print(f'The corresponding Seisbench Model Type is: {SBD_CLASS}')

In [ ]:
NUM_SAMPLES=3001            # Trace sample length - If the dataset is built for phasenet: 3001 If it is for EQTransformer: 6000
PHASE_LABEL = 'P' # 'S'     # What phase are we interested in. The SNR estimation will be based on its channel
SAMPLING_RATE = 100         # Sampling Rate - PhaseNet and EQTransformer expect 100Hz
SNR_THRESHOLD = 20          # A trace with SNR level above this threshold will be considered high
                            #                   and is a potential candidate to the high SNR dataset
BATCH_SIZE=256              # Number of traces that will be loaded at a single iteration
COMPARE_TO_TOTAL = False    # True if you want to keep all SNRs in memory and compare at the end. May be heavy for GEOFON e.g.

In [ ]:
SNR_CALC_STRATEGY_STR_ALTERNATIVES = ['energy_ratio', 'max_amplitude_vs_rms_ratio']
SNR_CALC_STRATEGY_STR =  'energy_ratio' # 'max_amplitude_vs_rms_ratio'
assert SNR_CALC_STRATEGY_STR in SNR_CALC_STRATEGY_STR_ALTERNATIVES, f'Expected one of {SNR_CALC_STRATEGY_STR_ALTERNATIVES}. Got {SNR_CALC_STRATEGY_STR}'
SNR_CALC_STRATEGY=str2snr_strategy(SNR_CALC_STRATEGY_STR)  # Function used to estimate the trace Signal to Noise Ratio(SNR) -  ENERGY_RATIO orMAX_AMPLITUDE_VS_RMS_RATIO
SNR_CALC_STRATEGY

Browse the path the generated dataset will be saved:

In [ ]:
SAVE_TO_PATH= os.path.join(DATASETS_ROOT_DIR,
                           f'noisy_datasets/{dataset_origin}_{NUM_SAMPLES}_sample_joachim_noises_{SNR_CALC_STRATEGY_STR}_snr/')
assert os.path.exists(SAVE_TO_PATH), f'Path {SAVE_TO_PATH} does not exist'
SAVE_TO_PATH

Browse the path targets file:

In [ ]:
TARGETS_PATH = os.path.join(DATASETS_ROOT_DIR, f'targets/{dataset_origin}/')
assert os.path.exists(TARGETS_PATH), f'Path {TARGETS_PATH} does not exist'
TARGETS_PATH

Uncomment relevant lines if downloading targets is needed

In [ ]:
# !mkdir targets
# !mkdir targets/ethz
# !mkdir targets/geofon
#
# #ethz
# !wget https://dcache-demo.desy.de:2443/Helmholtz/HelmholtzAI/SeisBench/auxiliary/pick-benchmark/targets/ethz/task1.csv
# !wget https://dcache-demo.desy.de:2443/Helmholtz/HelmholtzAI/SeisBench/auxiliary/pick-benchmark/targets/ethz/task23.csv
# !mv *.csv targets/ethz
#
# #geofon
# !wget https://dcache-demo.desy.de:2443/Helmholtz/HelmholtzAI/SeisBench/auxiliary/pick-benchmark/targets/geofon/task1.csv
# !wget https://dcache-demo.desy.de:2443/Helmholtz/HelmholtzAI/SeisBench/auxiliary/pick-benchmark/targets/geofon/task23.csv
# !mv *.csv targets/geofon

## Download Data and Load to Memory

### Load Dataset

In [ ]:
# Take only test data
tr, dv, ts = SBD_CLASS(sampling_rate=SAMPLING_RATE, force=True).train_dev_test()
PARTITIONS = {'train':tr, 'dev':dv, 'test':ts}

data = PARTITIONS[PARTITION_TO_USE]
print(data)

### Create Data Loader

In [ ]:
loader, num_dataset_traces = create_loader_by_phase_and_length(phase_label=PHASE_LABEL, trace_length=NUM_SAMPLES, targets_path=TARGETS_PATH, data=data, batch_size=BATCH_SIZE)
print(f'There are {num_dataset_traces} traces in the resulting dataset.')
print(f'Created a dataloader that loads all data in {len(loader)} batches')

## Extract The High SNR Traces

Load the whole traces batch by batch, calculate the traces snr and keep the traces with high snr.

In [ ]:
traces_counter = 0
SNR_of_high_SNR_traces_list, high_SNR_traces_list, high_SNR_traces_labels_list = [],[],[]
total_SNR = []
pbar =  tqdm(loader)
for dataset_dict in pbar:
    traces, labels = dataset_dict['X'], dataset_dict['onset_sample']
    SNR = calc_batch_snr(batch_traces=traces, batch_labels=labels, snr_estimation_strategy=SNR_CALC_STRATEGY)
    if COMPARE_TO_TOTAL:
        total_SNR.extend(SNR.tolist())
    batch_high_SNR_traces, batch_high_SNR_traces_labels, batch_SNR_of_high_SNR_traces = get_high_snr_traces(traces=traces, labels=labels, SNR=SNR, snr_threshold=SNR_THRESHOLD)
    traces_counter += batch_high_SNR_traces.shape[0]
    high_SNR_traces_list.append(batch_high_SNR_traces)
    high_SNR_traces_labels_list.append(batch_high_SNR_traces_labels.unsqueeze(dim=1))
    SNR_of_high_SNR_traces_list.append(batch_SNR_of_high_SNR_traces.unsqueeze(dim=1))

    pbar.set_description(f'{traces_counter} high SNR traces found')

SNR_of_high_SNR_traces = torch.vstack(SNR_of_high_SNR_traces_list).squeeze()
high_SNR_traces = torch.vstack(high_SNR_traces_list)
high_SNR_traces_labels = torch.vstack(high_SNR_traces_labels_list).squeeze()

print(f'Extracted high_SNR_traces.shape =  {high_SNR_traces.shape}')
print(f'SNR_of_high_SNR_traces.shape = {SNR_of_high_SNR_traces.shape}')
print(f'high_SNR_traces_labels.shape {high_SNR_traces_labels.shape}')

## Save the High SNR Dataset to Files

There are 3 files generated:
1. Traces
2. Corresponding labels
3. Corresponding SNR values

In [ ]:
torch.save(high_SNR_traces, os.path.join(SAVE_TO_PATH, 'original_dataset.pt'))
torch.save(high_SNR_traces_labels, os.path.join(SAVE_TO_PATH, 'original_labels.pt'))
torch.save(SNR_of_high_SNR_traces, os.path.join(SAVE_TO_PATH, f'original_traces_{SNR_CALC_STRATEGY_STR}_snrs.pt'))

In [ ]:
print(f'Saved to {SAVE_TO_PATH}')

## Plot the SNR values of the Extracted Dataset

In [ ]:
num_high_snr_traces = int(SNR_of_high_SNR_traces.shape[0])
if COMPARE_TO_TOTAL:
    fig, (ax_total_snr,ax_high_snr) = plt.subplots(1,2,figsize=(18, 6))
    plt.suptitle(f'SNR -  Estimated Using {SNR_CALC_STRATEGY_STR}')
    ax_high_snr.set_title('SNR (dB) Values Above Threshold')
    ax_high_snr.set_xticklabels([])
    ax_high_snr.set_xticks([])
    ax_high_snr.scatter(range(num_high_snr_traces), SNR_of_high_SNR_traces, s=1);
    if COMPARE_TO_TOTAL:
        ax_total_snr.set_title('SNR (dB) All Dataset')
        ax_total_snr.set_xticklabels([])
        ax_total_snr.set_xticks([])
        ax_total_snr.hlines(y=SNR_THRESHOLD, xmin=0, xmax=len(total_SNR), linestyles='dashed', colors='r')
        ax_total_snr.scatter(range(len(total_SNR)), total_SNR, s=1);
else:
    fig, ax_high_snr = plt.subplots(figsize=(8, 6))
    plt.suptitle(f'SNR -  Estimated Using {SNR_CALC_STRATEGY_STR}')
    ax_high_snr.set_title('SNR (dB) Values Above Threshold')
    ax_high_snr.set_xticklabels([])
    ax_high_snr.set_xticks([])
    ax_high_snr.scatter(range(num_high_snr_traces), SNR_of_high_SNR_traces, s=1);
